In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold, StratifiedKFold

In [3]:
df_tbi = pd.read_excel("H:\CTSN\Pour_revised_manuscripts\Data_TBI_after_process.xlsx")
df_tbi.head()

,Unnamed: 0,Age,Sex,Marshall (t0),Entry Diagnosis (t0),CRS-R (t1),RLAS (t1),DRS (t1),ERBI A (t1),ERBI B (t1),GOS-E (t2)
0,0,1.380828,0.630286,0.941067,0.673219,-0.823193,-0.585028,-0.15615,-0.295498,-0.251339,1
1,1,-0.960077,0.630286,-0.420477,0.673219,-1.080331,-0.585028,0.30551,0.381824,-0.251339,3
2,2,-1.350228,-1.586582,-1.101249,0.673219,-0.951762,-0.585028,-0.15615,-0.295498,-0.251339,3
3,3,-1.252690,0.630286,-1.101249,-0.869890,0.848201,0.043107,-0.84864,-0.295498,-0.251339,4
4,4,0.698064,0.630286,-1.101249,-0.869890,0.848201,0.043107,-0.84864,0.720485,1.030489,3


In [4]:
X = df_tbi[['Age', 'ERBI A (t1)', 'CRS-R (t1)', 'RLAS (t1)', 'DRS (t1)']]
X
y = df_tbi[['GOS-E (t2)']]

In [2]:
# Khởi tạo SMOTE và RandomForestClassifier
smote = SMOTE(random_state=42)
rf = RandomForestClassifier(n_estimators=200, random_state=42,max_depth=10, min_samples_split=2)
models = [rf]
model_names = ['Random Forest']

# Sử dụng StratifiedKFold để giữ nguyên tỷ lệ lớp
k_fold = StratifiedKFold(n_splits=10, shuffle=True)
skf =LeaveOneOut()

In [5]:
accuracy_scores = []
f1_scores = []
sensitivity_scores = []
specificity_scores = []
# Perform LOOCV
for train_index, test_index in k_fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
    # Apply SMOTE to the training set
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Iterate over each model
    for model in models:
        # Train the model
        model.fit(X_train_smote, y_train_smote)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        # Calculate sensitivity and specificity for each class
        sensitivity = []
        specificity = []
        for i in range(conf_matrix.shape[0]):
            tp = conf_matrix[i, i]
            fn = np.sum(conf_matrix[i, :]) - tp
            fp = np.sum(conf_matrix[:, i]) - tp
            tn = np.sum(conf_matrix) - tp - fn - fp
            
            sensitivity.append(tp / (tp + fn) if (tp + fn) != 0 else 0)
            specificity.append(tn / (tn + fp) if (tn + fp) != 0 else 0)
        
        # Append mean sensitivity and specificity to lists
        sensitivity_scores.append(np.mean(sensitivity))
        specificity_scores.append(np.mean(specificity))
        
        # Append results to lists
        accuracy_scores.append(accuracy)
        f1_scores.append(f1)

# Print results
for i, model in enumerate(models):
    print(f"Model: {model_names[i]}")
    print(f"Accuracy: {np.mean(accuracy_scores[i::len(models)]):.4f}")
    print(f"F1 Score: {np.mean(f1_scores[i::len(models)]):.4f}")
    print(f"Mean Sensitivity (Recall): {np.mean(sensitivity_scores[i::len(models)]):.4f}")
    print(f"Mean Specificity: {np.mean(specificity_scores[i::len(models)]):.4f}")
    print('---------------------------------------')

C:\Users\ad min\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\4292007806.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_smote, y_train_smote)
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\4292007806.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_smote, y_train_smote)
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\4292007806.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

Model: Random Forest
Accuracy: 0.7055
F1 Score: 0.6977
Mean Sensitivity (Recall): 0.6479
Mean Specificity: 0.8983
---------------------------------------


In [6]:
accuracy_scores = []
f1_scores = []
sensitivity_scores = []
specificity_scores = []
# Perform LOOCV
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
    # Apply SMOTE to the training set
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Iterate over each model
    for model in models:
        # Train the model
        model.fit(X_train_smote, y_train_smote)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        # Calculate sensitivity and specificity for each class
        sensitivity = []
        specificity = []
        for i in range(conf_matrix.shape[0]):
            tp = conf_matrix[i, i]
            fn = np.sum(conf_matrix[i, :]) - tp
            fp = np.sum(conf_matrix[:, i]) - tp
            tn = np.sum(conf_matrix) - tp - fn - fp
            
            sensitivity.append(tp / (tp + fn) if (tp + fn) != 0 else 0)
            specificity.append(tn / (tn + fp) if (tn + fp) != 0 else 0)
        
        # Append mean sensitivity and specificity to lists
        sensitivity_scores.append(np.mean(sensitivity))
        specificity_scores.append(np.mean(specificity))
        
        # Append results to lists
        accuracy_scores.append(accuracy)
        f1_scores.append(f1)

# Print results
for i, model in enumerate(models):
    print(f"Model: {model_names[i]}")
    print(f"Accuracy: {np.mean(accuracy_scores[i::len(models)]):.4f}")
    print(f"F1 Score: {np.mean(f1_scores[i::len(models)]):.4f}")
    print(f"Mean Sensitivity (Recall): {np.mean(sensitivity_scores[i::len(models)]):.4f}")
    print(f"Mean Specificity: {np.mean(specificity_scores[i::len(models)]):.4f}")
    print('---------------------------------------')

C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\3691604485.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_smote, y_train_smote)
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\3691604485.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_smote, y_train_smote)
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\3691604485.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_smote, y_train_smote)
C:\Users\ad min\AppData\Local\Temp\ipykernel_10976\3691604485.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for e

Model: Random Forest
Accuracy: 0.6961
F1 Score: 0.6961
Mean Sensitivity (Recall): 0.6961
Mean Specificity: 0.0000
---------------------------------------
